In [1]:
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core import Environment
from azureml.core import ScriptRunConfig
from azureml.core import Dataset
from azureml.train.hyperdrive import RandomParameterSampling, HyperDriveConfig, PrimaryMetricGoal, choice,GridParameterSampling


ws = Workspace.from_config()
datastore = ws.get_default_datastore()
experiment = Experiment(workspace=ws, name='RandomForest_Grid_Best_Hyp')

workspace = Workspace.from_config()

dataset = Dataset.get_by_name(workspace, name='rx20_all_models', version='latest')

config = ScriptRunConfig(
        source_directory='./src',
        script='RandomForest_GridSearch.py',
        compute_target='A00238622',
        arguments=[
            '--input-data', dataset.as_named_input('rx20_all_models')],
    )


env = Environment.from_conda_specification(
        name='sklearn-env',
        file_path='sklearn-env.yml'
    )
config.run_config.environment = env


params = GridParameterSampling(
    {
        '--n_estimators': choice(200),
        '--max_depth' : choice(32),
        '--max_features': choice(0.6),
        '--min_samples_split' : choice(5),
    }
)

# Configure hyperdrive settings
hyperdrive = HyperDriveConfig(run_config=config, 
                              hyperparameter_sampling=params, 
                              policy=None, 
                              primary_metric_name='median_relative_error_test', 
                              primary_metric_goal=PrimaryMetricGoal.MINIMIZE, 
                              max_total_runs=100, 
                              max_concurrent_runs=10)

run = experiment.submit(hyperdrive)
run.display_name = 'RandomForest_Optimized'
aml_url = run.get_portal_url()
print("Submitted to compute cluster. Click link below")
print("")
print(aml_url)

Submitted to compute cluster. Click link below

https://ml.azure.com/runs/HD_61d4273f-c856-40c2-bc21-88084e6d20b8?wsid=/subscriptions/ac5709b8-dd2b-46e3-98ae-bf53155e0e80/resourcegroups/nereva_rg/workspaces/e2r2s14mls0004&tid=13c728e0-bb0c-4cf7-8e10-5b327279d6d9
